# ResNet-50 + Keras Tuner로 CIFAR-10 분류 모델 최적화

이 노트북은 ResNet-50과 Keras Tuner를 활용하여 CIFAR-10 데이터셋의 분류 모델을 최적화하는 과정을 보여줍니다.


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras_tuner import RandomSearch

# 데이터셋 로드 및 전처리
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # 정규화
y_train, y_test = y_train.flatten(), y_test.flatten()  # (N, 1) -> (N,)

In [2]:
# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

In [5]:
# 모델 빌더 함수 정의 (Keras Tuner 용)
def build_model(hp):
    base_model = ResNet50(
        weights='imagenet',
        include_top=False, 
        input_shape=(32, 32, 3)
    )
    base_model.trainable = False 

    inputs = layers.Input(shape=(32, 32, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(
        units=hp.Choice('dense_units', values=[128, 256, 512]),
        activation='relu'
    )(x)
    x = layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1))(x)
    outputs = layers.Dense(10, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [6]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='resnet50_tuning',
    project_name='cifar10'g
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

tuner.search(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=20,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)

Trial 1 Complete [00h 17m 48s]
val_accuracy: 0.3615499883890152

Best val_accuracy So Far: 0.3615499883890152
Total elapsed time: 00h 17m 48s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
256               |512               |dense_units
0.2               |0.2               |dropout_rate
adam              |adam              |optimizer

Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 35ms/step - accuracy: 0.1214 - loss: 3.0339 - val_accuracy: 0.2450 - val_loss: 2.0297 - learning_rate: 0.0010
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 30ms/step - accuracy: 0.1875 - loss: 2.3022 - val_accuracy: 0.2849 - val_loss: 1.9616 - learning_rate: 0.0010
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.2178 - loss: 2.1170 - val_accuracy: 0.2779 - val_loss: 1.9387 - learning_rate: 0.0010
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 24s 31ms/step - accuracy: 0.2317 - loss: 2.0762 - val_accuracy: 0.3007 - val_loss: 1.9240 - learning_rate: 0.0010
Epoch 5/20
7

KeyboardInterrupt: 

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best dense_units: {best_hps.get('dense_units')}")
print(f"Best dropout_rate: {best_hps.get('dropout_rate')}")
print(f"Best optimizer: {best_hps.get('optimizer')}")

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=50,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
val_loss, val_accuracy = model.evaluate(x_test, y_test)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")